In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import Voronoi, voronoi_plot_2d
import time
import statsmodels.api as sm

In [ ]:
#set overall plot params
import matplotlib.pyplot as plt 
plt.rc('text', usetex=True)
plt.rc('font', family='Serif')

import matplotlib as mpl 
mpl.rcParams['figure.figsize'] = [10, 7]
mpl.rcParams['font.size'] = 27

mpl.rcParams['savefig.dpi'] = 150 
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'

mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True

mpl.rcParams['xtick.major.size'] = 10 
mpl.rcParams['xtick.minor.size'] = 5 

mpl.rcParams['ytick.major.size'] = 10 
mpl.rcParams['ytick.minor.size'] = 5 

mpl.rcParams['xtick.labelsize'] = 23
mpl.rcParams['ytick.labelsize'] = 23

In [ ]:
M92_path = '/home/mying/Desktop/M92_data/'

In [ ]:
#read empirical chi2
combined_std = 39
combined_mean = 3712

In [ ]:
#fix to the best fit distance and reddening
DM_fit = 14.71
Red_fit = 0.0

In [ ]:
#read chi2 results
os.chdir(M92_path + 'outchi2')
test_chi2_in_range = np.array([])
for file in os.listdir():
    dp = pd.read_csv(file)
    dp_fit = dp[(dp['dm'] == DM_fit) & (dp['red'] == Red_fit) & (dp['chi2'] <= combined_mean + 5*combined_std)]
    if len(dp_fit) != 0:
        dp_fit['MCnumber'] = int(file[2:])
        if len(test_chi2_in_range) == 0:
            test_chi2_in_range = dp_fit.to_numpy()
        else:
            test_chi2_in_range = np.concatenate((test_chi2_in_range,dp_fit.to_numpy()),axis=0)

In [ ]:
#read calibration star test results
dp = pd.read_csv(M92_path + 'mccdf_full.csv')
cdf = []
for i in range(len(test_chi2_in_range)):
#    if test_chi2_in_range[i][3] < combined_mean:
#        cdf.append(1)
#    else:
    for j in range(len(dp)):
        if dp['MCnumber'].values[j] == int(test_chi2_in_range[i][4]):
            cdf.append(1 - dp['cdf'].values[j])
total_pt = 0
for file in test_chi2_in_range:
    if file[3] >= np.mean(combined_mean):
        total_pt += 1
density = np.linspace(1,total_pt,total_pt)/total_pt

In [ ]:
len(test_chi2_in_range)

In [ ]:
#plot chi2 distribition
chi2 = np.array(test_chi2_in_range)[:,3]
chi2 = np.sort(chi2.astype(float))
chi2 = chi2[chi2 >= np.mean(combined_mean)]
plt.plot(chi2 - np.mean(combined_mean),density)
plt.xlabel("$\Delta \chi^2$")
plt.ylabel('cdf')
plt.tick_params(axis='x',direction="in")
plt.tick_params(axis='y',direction="in")
#plt.savefig(M92_path + 'plots\\delta_chi2.pdf', dpi=150);

In [ ]:
#calculate weight for each iso
weight = []
for i in range(len(test_chi2_in_range)):
    if test_chi2_in_range[i][3] <= combined_mean:
        weight.append(cdf[i])
    else:
        for j in range(len(chi2)):
            if chi2[j] == test_chi2_in_range[i][3]:
                weight.append(cdf[i]*(1 - density[j]))

In [ ]:
#find age and std
import math
def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights,axis=0)
    # Fast and numerically precise:
    variance = np.average((values-average)**2, weights=weights)
    return (average, math.sqrt(variance))
average, std = weighted_avg_and_std(test_chi2_in_range[:,0],np.array(weight))

In [ ]:
print(average,std)

In [ ]:
#read all the var files
os.chdir(M92_path + 'var')
var = []
for i in range(len(test_chi2_in_range)):
    dp = pd.read_fwf("newvarfeh230.{}".format(str(int(test_chi2_in_range[i,4]))),widths=[10,30],names=['value', 'name'])
    var.append(dp['value'].values)
var = np.array(var)

In [ ]:
#prepare data
data = test_chi2_in_range
data[:,[0,1,2]] = data[:,[1,2,0]]
for i in range(len(data)):
    data[i,3] = weight[i]

In [ ]:
#Read all var files
total_var = var.T[:-1].tolist()
#total_var.append(data[:,0].tolist())
#total_var.append(data[:,1].tolist())
total_var = np.array(total_var)

In [ ]:
#generate covariance matrix
m_cov = np.cov(total_var, aweights=data[:,-1])

In [ ]:
import statsmodels.api as sm

In [ ]:
mod_wls = sm.WLS(data[:,2], np.vstack([total_var, np.ones(len(data))]).T, weights=data[:,-1])
res_wls = mod_wls.fit()
print(res_wls.summary())

In [ ]:
#find total variance
retval = 0
for i in range(len(res_wls.params) - 1):
    for j in range(len(res_wls.params) - 1):
        if i == j:
            retval += np.std(total_var.T[:,i])**2 *res_wls.params[i]**2
        else:
            retval += res_wls.params[i]*res_wls.params[j]*m_cov[i][j]
np.sqrt(retval)

In [ ]:
#total variance without correlation
retval = 0
for i in range(len(res_wls.params) - 1):
    for j in range(len(res_wls.params) - 1):
        if i == j:
            retval += np.std(total_var.T[:,i])**2 *res_wls.params[i]**2
        else:
            retval += 0
np.sqrt(retval)

In [ ]:
#create lists of names and error greater than 5%
y = []
for i in range(20):
    y.append(np.abs(np.std(total_var.T[:,i]) *res_wls.params[i]/average *100))
y = np.array(y)
names = ['[Fe/H]','Primordial He', r"[$\alpha$/Fe]",'Mixing length','He diffusion', 'Heavy element diffusion','Surface boundary condition', 'Convective envelope overshoot', r'$p + p \to H_2 + e + \nu$', r'${ }^{3}He + { }^{3}He \to { }^{4}He + p + p$', r'${ }^{3}He + { }^{4}He \to { }^{2}H + \gamma$', r'${ }^{12}C + p \to { }^{13}N + \gamma$ ', r'${ }^{13}C + p \to { }^{14}N + \gamma$', r'${ }^{14}N + p \to { }^{15}O + \gamma$', r'${ }^{16}N + p \to { }^{17}F + \gamma$','Low T opacities', "High T opacities",r'Triple-$alpha$ coeff', 'Plasma neutrino loses', 'Conductive opacities']
others = 0
mask = []
for i in range(len(names)):
    if y[i] > 0.05:
#        print(names[i])
#        print(y[i])
        mask.append(i)
    else:
        others += y[i]**2
names_update = []
for i in mask:
    names_update.append(names[i])
names_update.append('Others')
y = y[mask].tolist()
y.append(np.sqrt(others))
y = np.array(y)

In [ ]:
np.sqrt(retval)/average

In [ ]:
#Plot
from matplotlib.offsetbox import AnchoredText
mpl.rcParams['figure.figsize'] = [14, 10]
mpl.rcParams['xtick.major.size'] = 0
mpl.rcParams['xtick.minor.size'] = 0
#y = np.abs(np.array(ages) - average)/average*100
x = np.arange(17)
fig, ax2 = plt.subplots(1,1)
# plot the same data on both axes
ax2.bar(x, height=y)
ax2.axhline(y=np.sqrt(retval)/average*100,linestyle='--',c='black')
#ax2.legend(prop={'size': 18})
ax2.set_ylim(0.0, 3.4)  # outliers only
#ax2.axes.xaxis.set_visible(False)
#ax2.text(0.95, 0.95, r"$\textup{Combined Error:} 5.3\%$", transform=ax.transAxes, fontsize=14,verticalalignment='top')
txt = ax2.text(1.8, 2.9, r"$\textup{Combined Error: } 3.22\%$", size=25, ha="center", color="black")

#anchored_text = AnchoredText("Test", loc=2)
#ax2.add_artist(anchored_text)
ax2.axes.xaxis.set_ticklabels([])
#ax2.set_xticks(x)
#ax2.set_xticklabels(names,rotation='vertical',fontsize=10)
rects = ax2.patches
labels = names_update
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax2.text(rect.get_x() + rect.get_width() / 2, height+0.10, label,
            ha='center', va='bottom',rotation='vertical',fontsize=20)
# hide the spines between ax and ax2
ax2.xaxis.tick_top()
ax2.xaxis.tick_bottom()
ax2.set_ylabel(r'Error on age ( $\%$ )',fontsize=22)
ax2.yaxis.set_label_coords(0.09, 0.5, transform=fig.transFigure)
#plt.savefig(M92_path + 'plots/error_budget.png', dpi=300)
plt.show();

In [ ]:
average, std = weighted_avg_and_std(test_chi2_in_range[:,2],np.array(weight))
#bins = np.linspace(11.7999,15.7999,21).tolist()
#bins.append(16.0001)
#bins = np.array(bins)
plt.hist(test_chi2_in_range[:,2]/1000,weights=weight,density=True)#,bins = bins)
#x_axis = np.linspace(11.6,16.2,240)
#plt.plot(x_axis, norm.pdf(x_axis, average/1000, std/1000),'--',label='Best-fit Gaussian')
#plt.errorbar(average/1000, 0.3, xerr=np.array([[average/1000 - 12400/1000], [15400/1000 - average/1000]]),capsize=5,label=r"95 \% CI",c='r')
#plt.scatter(average/1000, 0.3,s=50,c='r')
plt.xlabel('Age (Gyr)')
plt.ylabel('Density')
plt.legend(fontsize=18)
#plt.savefig(M92_path + 'plots\\Age_distribution.pdf', dpi=300);

In [ ]:
#fix to the best fit distance and reddening
DM_fit = 14.82
Red_fit = 0.0
#read chi2 results
os.chdir(M92_path + 'outchi2')
test_chi2_in_range = np.array([])
for file in os.listdir():
    dp = pd.read_csv(file)
    #dp_fit = dp[(dp['dm'] == DM_fit) & (dp['red'] == Red_fit) & (dp['chi2'] <= combined_mean + 5*combined_std)]
    dp_fit = dp[(dp['dm'] == DM_fit) & (dp['chi2'] <= combined_mean + 5*combined_std)]
    if len(dp_fit) != 0:
        dp_fit['MCnumber'] = int(file[2:])
        if len(test_chi2_in_range) == 0:
            test_chi2_in_range = dp_fit.to_numpy()
        else:
            test_chi2_in_range = np.concatenate((test_chi2_in_range,dp_fit.to_numpy()),axis=0)
#read calibration star test results
dp = pd.read_csv(M92_path + 'mccdf_full.csv')
cdf = []
for i in range(len(test_chi2_in_range)):
#    if test_chi2_in_range[i][3] < combined_mean:
#        cdf.append(1)
#    else:
    for j in range(len(dp)):
        if dp['MCnumber'].values[j] == int(test_chi2_in_range[i][4]):
            cdf.append(1 - dp['cdf'].values[j])
total_pt = 0
for file in test_chi2_in_range:
    if file[3] >= np.mean(combined_mean):
        total_pt += 1
density = np.linspace(1,total_pt,total_pt)/total_pt
chi2 = np.array(test_chi2_in_range)[:,3]
chi2 = np.sort(chi2.astype(float))
chi2 = chi2[chi2 >= np.mean(combined_mean)]
#calculate weight for each iso
weight = []
for i in range(len(test_chi2_in_range)):
    if test_chi2_in_range[i][3] <= combined_mean:
        weight.append(cdf[i])
    else:
        for j in range(len(chi2)):
            if chi2[j] == test_chi2_in_range[i][3]:
                weight.append(cdf[i]*(1 - density[j]))
#find age and std
import math
def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights,axis=0)
    # Fast and numerically precise:
    variance = np.average((values-average)**2, weights=weights)
    return (average, math.sqrt(variance))
average, std = weighted_avg_and_std(test_chi2_in_range[:,0],np.array(weight))
#read all the var files
os.chdir(M92_path + 'var')
var = []
for i in range(len(test_chi2_in_range)):
    dp = pd.read_fwf("newvarfeh230.{}".format(str(int(test_chi2_in_range[i,4]))),widths=[10,30],names=['value', 'name'])
    var.append(dp['value'].values)
var = np.array(var)
#prepare data
data = test_chi2_in_range
data[:,[0,1,2]] = data[:,[1,2,0]]
for i in range(len(data)):
    data[i,3] = weight[i]
#Read all var files
total_var = var.T[:-1].tolist()
total_var.append(data[:,0].tolist())
total_var.append(data[:,1].tolist())
total_var = np.array(total_var)
#generate covariance matrix
m_cov = np.cov(total_var, aweights=data[:,-1])
mod_wls = sm.WLS(data[:,2], np.vstack([total_var, np.ones(len(data))]).T, weights=data[:,-1])
res_wls = mod_wls.fit()
print(res_wls.summary())

In [ ]:
len(test_chi2_in_range)

In [ ]:
#find total variance
retval = 0
for i in range(len(res_wls.params) - 1):
    for j in range(len(res_wls.params) - 1):
        if i == j:
            retval += np.std(total_var.T[:,i])**2 *res_wls.params[i]**2
        else:
            retval += res_wls.params[i]*res_wls.params[j]*m_cov[i][j]
std = np.sqrt(retval)
print(std,std/average)

In [ ]:
#create lists of names and error greater than 5%
y = []
for i in range(22):
    y.append(np.abs(np.std(total_var.T[:,i]) *res_wls.params[i]/average *100))
y = np.array(y)
names = ['[Fe/H]','Primordial He', r"[$\alpha$/Fe]",'Mixing length','He diffusion', 'Heavy element diffusion','Surface boundary condition', 'Convective envelope overshoot', r'$p + p \to H_2 + e + \nu$', r'${ }^{3}He + { }^{3}He \to { }^{4}He + p + p$', r'${ }^{3}He + { }^{4}He \to { }^{2}H + \gamma$', r'${ }^{12}C + p \to { }^{13}N + \gamma$ ', r'${ }^{13}C + p \to { }^{14}N + \gamma$', r'${ }^{14}N + p \to { }^{15}O + \gamma$', r'${ }^{16}N + p \to { }^{17}F + \gamma$','Low T opacities', "High T opacities",r'Triple-$alpha$ coeff', 'Plasma neutrino loses', 'Conductive opacities','Distance','Reddening']
others = 0
mask = []
for i in range(len(names)):
    if y[i] > 0.05:
#        print(names[i])
#        print(y[i])
        mask.append(i)
    else:
        others += y[i]**2
names_update = []
for i in mask:
    names_update.append(names[i])
names_update.append('Others')
y = y[mask].tolist()
y.append(np.sqrt(others))
y = np.array(y)
#Plot
from matplotlib.offsetbox import AnchoredText
mpl.rcParams['figure.figsize'] = [14, 10]
mpl.rcParams['xtick.major.size'] = 0
mpl.rcParams['xtick.minor.size'] = 0
#y = np.abs(np.array(ages) - average)/average*100
x = np.arange(16)
fig, ax2 = plt.subplots(1,1)
# plot the same data on both axes
ax2.bar(x, height=y)
ax2.axhline(y=np.sqrt(retval)/average*100,linestyle='--',c='black')
#ax2.legend(prop={'size': 18})
ax2.set_ylim(0.0, 3.4)  # outliers only
#ax2.axes.xaxis.set_visible(False)
#ax2.text(0.95, 0.95, r"$\textup{Combined Error:} 5.3\%$", transform=ax.transAxes, fontsize=14,verticalalignment='top')
txt = ax2.text(1.8, 3.0, r"$\textup{Combined Error: } 2.92\%$", size=25, ha="center", color="black")
txt = ax2.text(12.8, 3.0, "Distance Modulus = {}".format(DM_fit), size=25, ha="center", color="black")

#anchored_text = AnchoredText("Test", loc=2)
#ax2.add_artist(anchored_text)
ax2.axes.xaxis.set_ticklabels([])
#ax2.set_xticks(x)
#ax2.set_xticklabels(names,rotation='vertical',fontsize=10)
rects = ax2.patches
labels = names_update
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax2.text(rect.get_x() + rect.get_width() / 2, height+0.10, label,
            ha='center', va='bottom',rotation='vertical',fontsize=20)
# hide the spines between ax and ax2
ax2.xaxis.tick_top()
ax2.xaxis.tick_bottom()
ax2.set_ylabel(r'Error on age ( $\%$ )',fontsize=22)
ax2.yaxis.set_label_coords(0.09, 0.5, transform=fig.transFigure)
plt.savefig(M92_path + 'plots/error_budget_dm_14_dot_82.png', dpi=300)
plt.show();

In [ ]:
os.chdir(M92_path + 'outchi2')
test_chi2_in_range_fit = np.array([])
for file in os.listdir():
    dp = pd.read_csv(file)
    dp_fit = dp[(dp['dm'] == DM_fit) & (dp['red'] == Red_fit) & (dp['chi2'] <= combined_mean + 5*combined_std)]
    if len(dp_fit) != 0:
        dp_fit['MCnumber'] = int(file[2:])
        if len(test_chi2_in_range_fit) == 0:
            test_chi2_in_range_fit = dp_fit.to_numpy()
        else:
            test_chi2_in_range_fit = np.concatenate((test_chi2_in_range_fit,dp_fit.to_numpy()),axis=0)
#read calibration star test results
dp = pd.read_csv(M92_path + 'mccdf_full.csv')
cdf = []
for i in range(len(test_chi2_in_range_fit)):
#    if test_chi2_in_range[i][3] < combined_mean:
#        cdf.append(1)
#    else:
    for j in range(len(dp)):
        if dp['MCnumber'].values[j] == int(test_chi2_in_range_fit[i][4]):
            cdf.append(1 - dp['cdf'].values[j])
total_pt = 0
for file in test_chi2_in_range_fit:
    if file[3] >= np.mean(combined_mean):
        total_pt += 1
density = np.linspace(1,total_pt,total_pt)/total_pt
chi2 = np.array(test_chi2_in_range_fit)[:,3]
chi2 = np.sort(chi2.astype(float))
chi2 = chi2[chi2 >= np.mean(combined_mean)]
#calculate weight for each iso
weight_fit = []
for i in range(len(test_chi2_in_range_fit)):
    if test_chi2_in_range_fit[i][3] <= combined_mean:
        weight_fit.append(cdf[i])
    else:
        for j in range(len(chi2)):
            if chi2[j] == test_chi2_in_range_fit[i][3]:
                weight_fit.append(cdf[i]*(1 - density[j]))

In [ ]:
mpl.rcParams['figure.figsize'] = [14, 10]
mpl.rcParams['font.size'] = 27

mpl.rcParams['savefig.dpi'] = 150 
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'

mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True

mpl.rcParams['xtick.major.size'] = 10 
mpl.rcParams['xtick.minor.size'] = 5 

mpl.rcParams['ytick.major.size'] = 10 
mpl.rcParams['ytick.minor.size'] = 5 

mpl.rcParams['xtick.labelsize'] = 23
mpl.rcParams['ytick.labelsize'] = 23
average, std = weighted_avg_and_std(test_chi2_in_range[:,2],np.array(weight))
bins = np.linspace(11.7999,15.7999,21).tolist()
bins.append(16.0001)
bins = np.array(bins)
plt.hist(test_chi2_in_range[:,2]/1000,weights=weight,density=True, bins = bins,alpha=0.7,label='DM = 14.82')
plt.hist(test_chi2_in_range_fit[:,0]/1000,weights=weight_fit,density=True, bins = bins,alpha=0.7,label='DM = 14.71')
#x_axis = np.linspace(11.6,16.2,240)
#plt.plot(x_axis, norm.pdf(x_axis, average/1000, std/1000),'--',label='Best-fit Gaussian')
#plt.errorbar(average/1000, 0.3, xerr=np.array([[average/1000 - 12400/1000], [15400/1000 - average/1000]]),capsize=5,label=r"95 \% CI",c='r')
#plt.scatter(average/1000, 0.3,s=50,c='r')
plt.xlabel('Age (Gyr)')
plt.ylabel('Density')
plt.legend(fontsize=18)
plt.savefig(M92_path + 'plots/age_distribution.png', dpi=300)

In [ ]:
test_chi2_in_range_fit

In [ ]:
#read chi2 results
os.chdir(M92_path + 'outchi2')
test_chi2_in_range = np.array([])
count = 0
for file in os.listdir():
    dp = pd.read_csv(file)
    #dp_fit = dp[(dp['dm'] == DM_fit) & (dp['red'] == Red_fit) & (dp['chi2'] <= combined_mean + 5*combined_std)]
    dp_fit = dp[dp['chi2'] <= combined_mean + 5*combined_std].sort_values(by='chi2')
    if len(dp_fit) != 0:
        count += 1
        # dp_fit['MCnumber'] = int(file[2:])
        # if len(test_chi2_in_range) == 0:
        #     test_chi2_in_range = dp_fit.to_numpy()
        # else:
        #     test_chi2_in_range = np.concatenate((test_chi2_in_range,dp_fit.to_numpy()),axis=0)
count

In [ ]:
#read chi2 results
os.chdir(M92_path + 'outchi2')
retval = 8234
test_chi2_less_4000 = np.array([])
for file in os.listdir()[:retval]:
    dp = pd.read_csv(file)
    for i in range(41):
        idx = dp.index[dp['chi2']==dp.loc[126*i:126*(i+1)].min(axis=0)[-1]]
        chi2 = dp['chi2'].values[idx]
        if chi2 < 4000:
            if len(test_chi2_less_4000) == 0:
                test_chi2_less_4000 = np.concatenate((dp.iloc[idx].values[0],np.array([int(file[-5:])])))
            else:
                test_chi2_less_4000 = np.concatenate((test_chi2_less_4000, np.concatenate((dp.iloc[idx].values[0],np.array([int(file[-5:])])))))

In [ ]:
#find chi2 in range
result = []
for i in range(len(test_chi2_less_4000)):
    if i % 5 == 0:
        result.append(test_chi2_less_4000[i:i+5])
chi2_less_4000 = np.array(result)
test_chi2_in_range = chi2_less_4000[chi2_less_4000[:,3]<(combined_mean + combined_std * 5)]

In [ ]:
len(test_chi2_in_range)

In [ ]:
#plot chi2 distribition
chi2 = np.array(test_chi2_in_range)[:,3]
chi2 = np.sort(chi2.astype(float))
chi2 = chi2[chi2 >= np.mean(combined_mean)]
plt.plot(chi2 - np.mean(combined_mean),density)
plt.xlabel("$\Delta \chi^2$")
plt.ylabel('cdf')
plt.tick_params(axis='x',direction="in")
plt.tick_params(axis='y',direction="in")
#plt.savefig(M92_path + 'plots\\delta_chi2.pdf', dpi=150);

In [ ]:
len(chi2)

In [ ]:
test_chi2_in_range = test_chi2_in_range[test_chi2_in_range[:,3].argsort()]
test_chi2_in_range = test_chi2_in_range[:1100]

In [ ]:
#read calibration star test results
dp = pd.read_csv(M92_path + 'mccdf_full.csv')
cdf = []
for i in range(len(test_chi2_in_range)):
#    if test_chi2_in_range[i][3] < combined_mean:
#        cdf.append(1)
#    else:
    for j in range(len(dp)):
        if dp['MCnumber'].values[j] == int(test_chi2_in_range[i][4]):
            cdf.append(1 - dp['cdf'].values[j])
total_pt = 0
for file in test_chi2_in_range:
    if file[3] >= np.mean(combined_mean):
        total_pt += 1
density = np.linspace(1,total_pt,total_pt)/total_pt
chi2 = np.array(test_chi2_in_range)[:,3]
chi2 = np.sort(chi2.astype(float))
chi2 = chi2[chi2 >= np.mean(combined_mean)]
#calculate weight for each iso
weight = []
for i in range(len(test_chi2_in_range)):
    if test_chi2_in_range[i][3] <= combined_mean:
        weight.append(cdf[i])
    else:
        for j in range(len(chi2)):
            if chi2[j] == test_chi2_in_range[i][3]:
                weight.append(cdf[i]*(1 - density[j]))

In [ ]:
bins = np.linspace(11.7999,15.7999,21).tolist()
bins.append(16.0001)
bins = np.array(bins)
plt.hist(test_chi2_in_range[:,0]/1000,weights=weight,density=True, bins = bins)
#x_axis = np.linspace(11.6,16.2,240)
#plt.plot(x_axis, norm.pdf(x_axis, average/1000, std/1000),'--',label='Best-fit Gaussian')
#plt.errorbar(average/1000, 0.3, xerr=np.array([[average/1000 - 12400/1000], [15400/1000 - average/1000]]),capsize=5,label=r"95 \% CI",c='r')
#plt.scatter(average/1000, 0.3,s=50,c='r')
plt.xlabel('Age (Gyr)')
plt.ylabel('Density')
plt.legend(fontsize=18)
#plt.savefig(M92_path + 'plots/age_distribution.png', dpi=300)

In [ ]:
test_chi2_in_range[:,0]

In [ ]:
len(weight)

In [ ]:
d = {'Age (Gyr)': test_chi2_in_range[:,0]/1000, 'Weight': np.array(weight)}
df = pd.DataFrame(data=d)
df.to_csv("{}/Figure9_data.csv".format(M92_path),index=False)